In [ ]:
# Cell 1: Import necessary libraries
import pandas as pd
import re
import numpy as np # Import numpy
try:
    from tqdm.auto import tqdm  # For progress bars
    TQDM_AVAILABLE = True
except ImportError:
    TQDM_AVAILABLE = False
    print("Warning: tqdm library not found. Progress bars will not be shown. To enable them, please install tqdm: pip install tqdm")
    # Define a dummy tqdm if not available so the code doesn't break
    def tqdm(iterable, *args, **kwargs):
        return iterable


# Cell 2: Load the Excel files
# Replace 'extract.xlsx' and 'gold.xlsx' with the actual file paths
try:
    df_extract = pd.read_excel('extract.xlsx')
    df_gold = pd.read_excel('gold.xlsx')
    print("Successfully loaded 'extract.xlsx' and 'gold.xlsx'.")
except FileNotFoundError:
    print("One or both Excel files not found. Using dummy data for demonstration.")
    # Create dummy dataframes for demonstration if files are not found
    data_extract = {
        'filepath': ['/tmp/job_e6b7dd30-4f60-4f8b-8331-2458fea95c06_eh6vxyz3/zip_0_new/GenAI-OUT-F-IMG-16042025-30000/2510600056001201252_f.jpeg',
                     '/tmp/job_e6b7dd30-4f60-4f8b-8331-2458fea95c06_eh6vxyz3/zip_0_new/GenAI-OUT-F-IMG-16042025-30000/1234567890123456789_f.jpeg',
                     '/tmp/job_e6b7dd30-4f60-4f8b-8331-2458fea95c06_eh6vxyz3/zip_0_new/GenAI-OUT-F-IMG-16042025-30000/9999999999999999999_f.jpeg',
                     '/tmp/job_e6b7dd30-4f60-4f8b-8331-2458fea95c06_eh6vxyz3/zip_0_new/GenAI-OUT-F-IMG-16042025-30000/0000000000000000000_f.jpeg'], # Added a no-match record
        'payee_name': ['John Doe!', 'Jane Smith.', 'Another Person', 'No Match Name'],
        'amount_numeric': [750, 1200.50, 500, 100],
        'micr_code': ['123456CITYBANKBRANCH001SAVINGS', '987654CITY2BANK2BRANCH2002CURRENT', '111222CITY3BANK3BRANCH3003SAVINGS', 'NOMATCHMICR']
    }
    df_extract = pd.DataFrame(data_extract)

    data_gold = {
        'TRANSACTION_DATE': ['16/04/2025', '17/04/2025', '18/04/2025'],
        'INSTRUMENT_ID': [2510600056001201252, 9876543210987654321, 9999999999999999999],
        'FLOW_TYPE': ['INWARD', 'OUTWARD', 'INWARD'],
        'SCAN_INSTRUMENT_NUMBER': [123456, 987654, 111222],
        'SCAN_PAYEE_BANK_CITY_CODE': ['CITY', 'CITY2', 'CITY3'],
        'SCAN_PAYEE_BANK_CODE': ['BANK', 'BANK2', 'BANK3'],
        'SCAN_PAYEE_BANK_BRANCH_CODE': ['BRANCH', 'BRANCH2', 'BRANCH3'],
        'SCAN_MICR_ACNO': [1, 0, 3],
        'SCAN_INSTRUMENT_TYPE': ['SAVINGS', 'CURRENT', 'SAVINGS'],
        'PRES_NAME': [' John Doe ', 'Jane  Smith', 'Another Person '],
        'CAR_AMOUNT': [750.00, 1250.00, 500.00]
    }
    df_gold = pd.DataFrame(data_gold)

# Display the first few rows of each dataframe
print("\nExcel 1 (Extract) Head:")
print(df_extract.head())
print("\nExcel 2 (Gold) Head:")
print(df_gold.head())

# Cell 3: Preprocess df_extract
# Extract INSTRUMENT_ID from filepath
def extract_instrument_id(filepath):
    if pd.isna(filepath):
        return None
    match = re.search(r'/([^/]+)_f\.jpeg$', str(filepath))
    if match:
        return match.group(1)
    return None

df_extract['INSTRUMENT_ID_EXTRACT'] = df_extract['filepath'].apply(extract_instrument_id)
df_extract['INSTRUMENT_ID_EXTRACT'] = pd.to_numeric(df_extract['INSTRUMENT_ID_EXTRACT'], errors='coerce')

print("\nDF Extract after INSTRUMENT_ID extraction:")
print(df_extract.head())

# Cell 4: Preprocess df_gold
# Construct micr_code in df_gold
def construct_micr(row):
    parts = []
    if pd.notna(row['SCAN_INSTRUMENT_NUMBER']):
        parts.append(str(row['SCAN_INSTRUMENT_NUMBER']))
    if pd.notna(row['SCAN_PAYEE_BANK_CITY_CODE']):
        parts.append(str(row['SCAN_PAYEE_BANK_CITY_CODE']))
    if pd.notna(row['SCAN_PAYEE_BANK_CODE']):
        parts.append(str(row['SCAN_PAYEE_BANK_CODE']))
    if pd.notna(row['SCAN_PAYEE_BANK_BRANCH_CODE']):
        parts.append(str(row['SCAN_PAYEE_BANK_BRANCH_CODE']))
    if pd.notna(row['SCAN_MICR_ACNO']) and row['SCAN_MICR_ACNO'] != 0:
        parts.append(str(int(row['SCAN_MICR_ACNO'])))
    if pd.notna(row['SCAN_INSTRUMENT_TYPE']):
        parts.append(str(row['SCAN_INSTRUMENT_TYPE']))
    return "".join(parts)

df_gold['micr_code_gold'] = df_gold.apply(construct_micr, axis=1)
df_gold['INSTRUMENT_ID'] = pd.to_numeric(df_gold['INSTRUMENT_ID'], errors='coerce')

print("\nDF Gold after micr_code_gold construction:")
print(df_gold.head())

# Cell 5: Merge the DataFrames
df_merged = pd.merge(df_extract, df_gold, left_on='INSTRUMENT_ID_EXTRACT', right_on='INSTRUMENT_ID', how='left', suffixes=('_extract', '_gold'))

print("\nMerged DataFrame Head:")
print(df_merged.head())

# Cell 6: Define comparison functions
def compare_names(name1, name2):
    if pd.isna(name1) or pd.isna(name2):
        return False, "One or both names are NaN"
    norm_name1 = str(name1).strip().lower()
    norm_name2 = str(name2).strip().lower()
    norm_name1 = re.sub(r'[^a-z0-9\s]', '', norm_name1)
    norm_name2 = re.sub(r'[^a-z0-9\s]', '', norm_name2)
    norm_name1 = re.sub(r'\s+', ' ', norm_name1).strip()
    norm_name2 = re.sub(r'\s+', ' ', norm_name2).strip()
    return norm_name1 == norm_name2, f"Normalized Extract: '{norm_name1}', Normalized Gold: '{norm_name2}'"

def compare_amounts(amount1, amount2):
    if pd.isna(amount1) or pd.isna(amount2):
        return False
    try:
        return float(amount1) == float(amount2)
    except ValueError:
        return False

def compare_micr(micr1, micr2):
    if pd.isna(micr1) or pd.isna(micr2):
        return False
    return str(micr1).strip() == str(micr2).strip()

# Cell 7: Apply comparison functions and calculate match scores
if TQDM_AVAILABLE:
    tqdm.pandas(desc="Processing records for comparison") 

df_merged['payee_name_match_status'] = False
df_merged['payee_name_match_details'] = ''
df_merged['amount_match_status'] = False
df_merged['micr_match_status'] = False
df_merged['fields_matched'] = 0

print("\nApplying comparisons:")
# Use tqdm directly on iterrows
for index, row in tqdm(df_merged.iterrows(), total=df_merged.shape[0], desc="Comparing records"):
    current_matches = 0
    # Compare Payee Name
    if 'payee_name' in df_merged.columns and 'PRES_NAME' in df_merged.columns:
        if pd.notna(row['payee_name']) and pd.notna(row['PRES_NAME']):
            name_match_result, name_match_detail_str = compare_names(row['payee_name'], row['PRES_NAME'])
            df_merged.loc[index, 'payee_name_match_status'] = name_match_result
            df_merged.loc[index, 'payee_name_match_details'] = name_match_detail_str
            if name_match_result:
                current_matches += 1
        else:
             df_merged.loc[index, 'payee_name_match_details'] = "One or both names missing for comparison"
    else:
        df_merged.loc[index, 'payee_name_match_details'] = "Payee name columns not found"

    # Compare Amount
    if 'amount_numeric' in df_merged.columns and 'CAR_AMOUNT' in df_merged.columns:
        if pd.notna(row['amount_numeric']) and pd.notna(row['CAR_AMOUNT']):
            amount_match_result = compare_amounts(row['amount_numeric'], row['CAR_AMOUNT'])
            df_merged.loc[index, 'amount_match_status'] = amount_match_result
            if amount_match_result:
                current_matches += 1
    
    # Compare MICR
    if 'micr_code' in df_merged.columns and 'micr_code_gold' in df_merged.columns:
        if pd.notna(row['micr_code']) and pd.notna(row['micr_code_gold']):
            micr_match_result = compare_micr(row['micr_code'], row['micr_code_gold'])
            df_merged.loc[index, 'micr_match_status'] = micr_match_result
            if micr_match_result:
                current_matches += 1
    df_merged.loc[index, 'fields_matched'] = current_matches

fields_to_compare_list = []
for index, row in df_merged.iterrows():
    count = 0
    if 'payee_name' in df_merged.columns and 'PRES_NAME' in df_merged.columns and \
       pd.notna(row['payee_name']) and pd.notna(row['PRES_NAME']):
        count += 1
    if 'amount_numeric' in df_merged.columns and 'CAR_AMOUNT' in df_merged.columns and \
       pd.notna(row['amount_numeric']) and pd.notna(row['CAR_AMOUNT']):
        count += 1
    if 'micr_code' in df_merged.columns and 'micr_code_gold' in df_merged.columns and \
       pd.notna(row['micr_code']) and pd.notna(row['micr_code_gold']):
        count += 1
    fields_to_compare_list.append(count)
df_merged['fields_to_compare'] = fields_to_compare_list

# Calculate match percentage using np.where to avoid division by zero issues and recursion
df_merged['match_percentage'] = np.where(
    df_merged['fields_to_compare'] > 0,
    (df_merged['fields_matched'] / df_merged['fields_to_compare']) * 100,
    0  # Set to 0 if fields_to_compare is 0
)
df_merged['match_percentage'] = df_merged['match_percentage'].fillna(0) # Ensure any other NaNs become 0


print("\nDataFrame with Match Status and Percentage:")
print(df_merged[['INSTRUMENT_ID_EXTRACT', 'payee_name_match_status', 'amount_match_status', 'micr_match_status',
                 'fields_matched', 'fields_to_compare', 'match_percentage']].head())

# Cell 8: Select relevant columns for the output and save to Excel
output_columns = [
    'filepath', 'INSTRUMENT_ID_EXTRACT', 'payee_name', 'amount_numeric', 'micr_code', 
    'INSTRUMENT_ID', 'PRES_NAME', 'CAR_AMOUNT', 'micr_code_gold', 
    'SCAN_INSTRUMENT_NUMBER', 'SCAN_PAYEE_BANK_CITY_CODE', 'SCAN_PAYEE_BANK_CODE', 
    'SCAN_PAYEE_BANK_BRANCH_CODE', 'SCAN_MICR_ACNO', 'SCAN_INSTRUMENT_TYPE', 
    'payee_name_match_status', 'payee_name_match_details', 'amount_match_status', 'micr_match_status', 
    'fields_matched', 'fields_to_compare', 'match_percentage' 
]

final_output_columns = [col for col in output_columns if col in df_merged.columns]
df_output = df_merged[final_output_columns]

try:
    output_file_name = 'comparison_results_final.xlsx'
    df_output.to_excel(output_file_name, index=False)
    print(f"\nComparison results saved to '{output_file_name}'")
except Exception as e:
    print(f"Error saving to Excel: {e}")

# Cell 9: Display overall statistics (optional)
total_records = len(df_merged)
# For perfect matches, consider only rows where there was something to compare
perfect_matches_df = df_merged[df_merged['fields_to_compare'] > 0]
perfect_matches = len(perfect_matches_df[perfect_matches_df['match_percentage'] == 100])

overall_field_matches_sum = df_merged['fields_matched'].sum()
overall_fields_to_compare_sum = df_merged['fields_to_compare'].sum()

overall_match_accuracy = (overall_field_matches_sum / overall_fields_to_compare_sum * 100) if overall_fields_to_compare_sum > 0 else 0

print(f"\n--- Overall Statistics ---")
print(f"Total Records Processed: {total_records}")
print(f"Records with 100% Match (among comparable): {perfect_matches}")

if overall_fields_to_compare_sum > 0:
    print(f"Overall Field Match Accuracy: {overall_match_accuracy:.2f}% ({overall_field_matches_sum}/{overall_fields_to_compare_sum} fields)")
else:
    print("Overall Field Match Accuracy: Not applicable (no fields to compare or all comparable fields were NaN)")

if 'payee_name_match_status' in df_merged.columns:
    name_matches = df_merged['payee_name_match_status'].sum()
    name_total_comparable = len(df_merged[df_merged['payee_name'].notna() & df_merged['PRES_NAME'].notna()])
    name_accuracy = (name_matches / name_total_comparable * 100) if name_total_comparable > 0 else 0
    print(f"Payee Name Match Accuracy: {name_accuracy:.2f}% ({name_matches}/{name_total_comparable} comparable pairs)")

if 'amount_match_status' in df_merged.columns:
    amount_matches = df_merged['amount_match_status'].sum()
    amount_total_comparable = len(df_merged[df_merged['amount_numeric'].notna() & df_merged['CAR_AMOUNT'].notna()])
    amount_accuracy = (amount_matches / amount_total_comparable * 100) if amount_total_comparable > 0 else 0
    print(f"Amount Match Accuracy: {amount_accuracy:.2f}% ({amount_matches}/{amount_total_comparable} comparable pairs)")

if 'micr_match_status' in df_merged.columns:
    micr_matches = df_merged['micr_match_status'].sum()
    micr_total_comparable = len(df_merged[df_merged['micr_code'].notna() & df_merged['micr_code_gold'].notna()])
    micr_accuracy = (micr_matches / micr_total_comparable * 100) if micr_total_comparable > 0 else 0
    print(f"MICR Code Match Accuracy: {micr_accuracy:.2f}% ({micr_matches}/{micr_total_comparable} comparable pairs)")

print("--- End of Notebook ---")

In [ ]:
# Cell 1: Import necessary libraries
import pandas as pd
import re
import numpy as np # Import numpy
try:
    from tqdm.auto import tqdm  # For progress bars
    tqdm.pandas() # Initialize tqdm for pandas functions like progress_apply
    TQDM_AVAILABLE = True
except ImportError:
    TQDM_AVAILABLE = False
    print("Warning: tqdm library not found. Progress bars for apply functions will not be shown. To enable them, please install tqdm: pip install tqdm")
    # Define a dummy tqdm if not available so the code doesn't break
    def tqdm(iterable, *args, **kwargs): # This dummy is for iterables
        return iterable
    # If tqdm.pandas() can't be called, progress_apply will just be apply
    # We can create a dummy progress_apply if needed, or let it default to apply
    # For simplicity, if tqdm is not there, progress_apply will become a direct apply call later

# Cell 2: Load the Excel files
# Replace 'extract.xlsx' and 'gold.xlsx' with the actual file paths
try:
    df_extract = pd.read_excel('extract.xlsx')
    df_gold = pd.read_excel('gold.xlsx')
    print("Successfully loaded 'extract.xlsx' and 'gold.xlsx'.")
except FileNotFoundError:
    print("One or both Excel files not found. Using dummy data for demonstration (30k rows).")
    # Create larger dummy dataframes for demonstration
    num_records = 30000
    base_instrument_id = 2510600056001201252
    data_extract = {
        'filepath': [f'/tmp/job/path/to/{base_instrument_id + i}_f.jpeg' for i in range(num_records)],
        'payee_name': [f'Payee Name {i % 100}!' if i % 2 == 0 else f' Payee Name {i % 100} ' for i in range(num_records)],
        'amount_numeric': [750 + (i%1000) if i % 2 == 0 else (750 + (i%1000)) * 1.0 for i in range(num_records)],
        'micr_code': [f'123456CITYBANKBRANCH00{i%10}SAVINGS' if i%3==0 else f'654321CITYBANKALT0{i%5}CURRENT' for i in range(num_records)]
    }
    df_extract = pd.DataFrame(data_extract)
    # Add some NaNs for testing robustness
    for col in ['payee_name', 'amount_numeric', 'micr_code']:
      df_extract.loc[df_extract.sample(frac=0.05).index, col] = np.nan


    data_gold = {
        'TRANSACTION_DATE': [f'{(i%28)+1}/{(i%12)+1}/2025' for i in range(num_records)],
        'INSTRUMENT_ID': [base_instrument_id + i for i in range(num_records)],
        'FLOW_TYPE': ['INWARD' if i % 2 == 0 else 'OUTWARD' for i in range(num_records)],
        'SCAN_INSTRUMENT_NUMBER': [123456 + i for i in range(num_records)],
        'SCAN_PAYEE_BANK_CITY_CODE': ['CITY' if i%2==0 else 'CITYALT' for i in range(num_records)],
        'SCAN_PAYEE_BANK_CODE': ['BANK' for i in range(num_records)],
        'SCAN_PAYEE_BANK_BRANCH_CODE': ['BRANCH' if i%2==0 else 'BRANCHALT' for i in range(num_records)],
        'SCAN_MICR_ACNO': [i % 5 if (i % 5 != 0) else (0 if i%10==0 else np.nan) for i in range(num_records)], # include 0 and NaN
        'SCAN_INSTRUMENT_TYPE': ['SAVINGS' if i%3==0 else 'CURRENT' for i in range(num_records)],
        'PRES_NAME': [f'Payee Name {i % 100}' for i in range(num_records)], # Gold standard names
        'CAR_AMOUNT': [750 + (i%1000) for i in range(num_records)]
    }
    df_gold = pd.DataFrame(data_gold)
    # Add some NaNs for testing robustness
    for col in ['PRES_NAME', 'CAR_AMOUNT', 'SCAN_MICR_ACNO']:
      df_gold.loc[df_gold.sample(frac=0.05).index, col] = np.nan


print(f"\nExcel 1 (Extract) Head ({len(df_extract)} rows):")
print(df_extract.head())
print(f"\nExcel 2 (Gold) Head ({len(df_gold)} rows):")
print(df_gold.head())

# Cell 3: Preprocess df_extract (Vectorized where possible)
print("\nPreprocessing df_extract...")
df_extract['INSTRUMENT_ID_EXTRACT'] = df_extract['filepath'].astype(str).str.extract(r'/([^/]+)_f\.jpeg$')[0]
df_extract['INSTRUMENT_ID_EXTRACT'] = pd.to_numeric(df_extract['INSTRUMENT_ID_EXTRACT'], errors='coerce')
print("DF Extract after INSTRUMENT_ID extraction head:")
print(df_extract.head())

# Cell 4: Preprocess df_gold (Vectorized MICR construction)
print("\nPreprocessing df_gold (vectorized MICR)...")

# Convert components to string, filling NaNs with empty strings
s_instr_num = df_gold['SCAN_INSTRUMENT_NUMBER'].fillna('').astype(str)
s_city = df_gold['SCAN_PAYEE_BANK_CITY_CODE'].fillna('').astype(str)
s_bank = df_gold['SCAN_PAYEE_BANK_CODE'].fillna('').astype(str)
s_branch = df_gold['SCAN_PAYEE_BANK_BRANCH_CODE'].fillna('').astype(str)
s_instr_type = df_gold['SCAN_INSTRUMENT_TYPE'].fillna('').astype(str)

# Conditional SCAN_MICR_ACNO
# Ensure it's numeric before comparison, then convert to int string, or empty
micr_acno_num = pd.to_numeric(df_gold['SCAN_MICR_ACNO'], errors='coerce')
s_micr_acno = np.where(
    (micr_acno_num.notna()) & (micr_acno_num != 0),
    micr_acno_num.astype(float).astype(int).astype(str), # astype(float) handles cases like "1.0" before int
    ''
)
df_gold['micr_code_gold'] = s_instr_num + s_city + s_bank + s_branch + s_micr_acno + s_instr_type
df_gold['INSTRUMENT_ID'] = pd.to_numeric(df_gold['INSTRUMENT_ID'], errors='coerce')
print("DF Gold after micr_code_gold construction head:")
print(df_gold[['INSTRUMENT_ID', 'micr_code_gold']].head())

# Cell 5: Merge the DataFrames
print("\nMerging DataFrames...")
df_merged = pd.merge(df_extract, df_gold, left_on='INSTRUMENT_ID_EXTRACT', right_on='INSTRUMENT_ID', how='left', suffixes=('_extract', '_gold'))
print(f"Merged DataFrame shape: {df_merged.shape}")
print("Merged DataFrame Head:")
print(df_merged.head())

# Cell 6: Comparison logic (will be applied vectorially in Cell 7)
# Normalization function for names (can be applied to series)
def normalize_name_series(name_series):
    if not isinstance(name_series, pd.Series):
        name_series = pd.Series(name_series)
    norm_s = name_series.astype(str).str.strip().str.lower()
    norm_s = norm_s.str.replace(r'[^a-z0-9\s]', '', regex=True)
    norm_s = norm_s.str.replace(r'\s+', ' ', regex=True).str.strip()
    return norm_s

# Cell 7: Apply comparisons (Vectorized) and calculate match scores
print("\nApplying comparisons (vectorized)...")

# --- 1. Payee Name Comparison ---
name1_norm = normalize_name_series(df_merged['payee_name'])
name2_norm = normalize_name_series(df_merged['PRES_NAME'])
name_comparable = df_merged['payee_name'].notna() & df_merged['PRES_NAME'].notna()
df_merged['payee_name_match_status'] = name_comparable & (name1_norm == name2_norm)

# For payee_name_match_details (using apply for rich string, this is the slowest part but acceptable for one col)
def generate_name_details(row, norm_s1_col, norm_s2_col):
    idx = row.name # get the original index
    is_comparable = row['name_comparable_temp'] # Use temporary column passed via closure/context
    
    if not is_comparable:
        return "One or both names are NaN"
    
    n1 = norm_s1_col.loc[idx]
    n2 = norm_s2_col.loc[idx]
    
    if row['payee_name_match_status']:
        return f"Normalized Match: Ext: '{n1}', Gold: '{n2}'"
    else:
        return f"Normalized No Match: Ext: '{n1}', Gold: '{n2}'"

df_merged['name_comparable_temp'] = name_comparable # Create temporary column for apply access
print("Generating payee name match details (this may take a moment for 30k rows)...")
if TQDM_AVAILABLE:
    df_merged['payee_name_match_details'] = df_merged.progress_apply(
        lambda row: generate_name_details(row, name1_norm, name2_norm), axis=1
    )
else:
    df_merged['payee_name_match_details'] = df_merged.apply(
        lambda row: generate_name_details(row, name1_norm, name2_norm), axis=1
    )
del df_merged['name_comparable_temp'] # cleanup

# --- 2. Amount Comparison ---
amount1_num = pd.to_numeric(df_merged['amount_numeric'], errors='coerce')
amount2_num = pd.to_numeric(df_merged['CAR_AMOUNT'], errors='coerce')
amount_comparable = amount1_num.notna() & amount2_num.notna()
# Compare as floats for precision (e.g., 750 == 750.0)
df_merged['amount_match_status'] = amount_comparable & (amount1_num.astype(float) == amount2_num.astype(float))

# --- 3. MICR Code Comparison ---
micr1_norm = df_merged['micr_code'].astype(str).str.strip()
micr2_norm = df_merged['micr_code_gold'].astype(str).str.strip() # Already constructed
micr_comparable = df_merged['micr_code'].notna() & df_merged['micr_code_gold'].notna() # Gold MICR can be empty if all parts NaN
df_merged['micr_match_status'] = micr_comparable & (micr1_norm == micr2_norm)

# --- 4. Calculate Fields Matched and Fields to Compare ---
df_merged['fields_to_compare'] = name_comparable.astype(int) + \
                                 amount_comparable.astype(int) + \
                                 micr_comparable.astype(int)

df_merged['fields_matched'] = df_merged['payee_name_match_status'].astype(int) + \
                              df_merged['amount_match_status'].astype(int) + \
                              df_merged['micr_match_status'].astype(int)

# --- 5. Calculate Match Percentage ---
df_merged['match_percentage'] = np.where(
    df_merged['fields_to_compare'] > 0,
    (df_merged['fields_matched'] / df_merged['fields_to_compare']) * 100,
    0  # Set to 0 if fields_to_compare is 0
).round(2) # Round to 2 decimal places

print("\nDataFrame with Match Status and Percentage (first 5 rows):")
print(df_merged[['INSTRUMENT_ID_EXTRACT', 'payee_name_match_status', 'amount_match_status', 'micr_match_status',
                 'fields_matched', 'fields_to_compare', 'match_percentage', 'payee_name_match_details']].head())

# Cell 8: Select relevant columns for the output and save to Excel
print("\nPreparing output file...")
output_columns = [
    'filepath', 'INSTRUMENT_ID_EXTRACT', 'payee_name', 'amount_numeric', 'micr_code', 
    'INSTRUMENT_ID', 'PRES_NAME', 'CAR_AMOUNT', 'micr_code_gold', 
    'SCAN_INSTRUMENT_NUMBER', 'SCAN_PAYEE_BANK_CITY_CODE', 'SCAN_PAYEE_BANK_CODE', 
    'SCAN_PAYEE_BANK_BRANCH_CODE', 'SCAN_MICR_ACNO', 'SCAN_INSTRUMENT_TYPE', 
    'payee_name_match_status', 'payee_name_match_details', 'amount_match_status', 'micr_match_status', 
    'fields_matched', 'fields_to_compare', 'match_percentage' 
]

final_output_columns = [col for col in output_columns if col in df_merged.columns]
df_output = df_merged[final_output_columns]

try:
    output_file_name = 'comparison_results_vectorized.xlsx'
    df_output.to_excel(output_file_name, index=False)
    print(f"\nComparison results saved to '{output_file_name}'")
except Exception as e:
    print(f"Error saving to Excel: {e}")

# Cell 9: Display overall statistics (optional)
print("\nCalculating overall statistics...")
total_records = len(df_merged)
perfect_matches_df = df_merged[df_merged['fields_to_compare'] > 0]
perfect_matches = len(perfect_matches_df[perfect_matches_df['match_percentage'] == 100])

overall_field_matches_sum = df_merged['fields_matched'].sum()
overall_fields_to_compare_sum = df_merged['fields_to_compare'].sum()

overall_match_accuracy = (overall_field_matches_sum / overall_fields_to_compare_sum * 100) if overall_fields_to_compare_sum > 0 else 0

print(f"\n--- Overall Statistics ---")
print(f"Total Records Processed: {total_records}")
print(f"Records with 100% Match (among comparable): {perfect_matches}")

if overall_fields_to_compare_sum > 0:
    print(f"Overall Field Match Accuracy: {overall_match_accuracy:.2f}% ({overall_field_matches_sum}/{overall_fields_to_compare_sum} fields)")
else:
    print("Overall Field Match Accuracy: Not applicable (no fields to compare or all comparable fields were NaN)")

if 'payee_name_match_status' in df_merged.columns:
    name_matches = df_merged['payee_name_match_status'].sum() # Sum of True booleans
    # name_total_comparable is the sum of the 'name_comparable' boolean Series
    name_total_comparable_count = name_comparable.sum()
    name_accuracy = (name_matches / name_total_comparable_count * 100) if name_total_comparable_count > 0 else 0
    print(f"Payee Name Match Accuracy: {name_accuracy:.2f}% ({name_matches}/{name_total_comparable_count} comparable pairs)")

if 'amount_match_status' in df_merged.columns:
    amount_matches = df_merged['amount_match_status'].sum()
    amount_total_comparable_count = amount_comparable.sum()
    amount_accuracy = (amount_matches / amount_total_comparable_count * 100) if amount_total_comparable_count > 0 else 0
    print(f"Amount Match Accuracy: {amount_accuracy:.2f}% ({amount_matches}/{amount_total_comparable_count} comparable pairs)")

if 'micr_match_status' in df_merged.columns:
    micr_matches = df_merged['micr_match_status'].sum()
    micr_total_comparable_count = micr_comparable.sum() # micr_comparable was the boolean Series for this
    micr_accuracy = (micr_matches / micr_total_comparable_count * 100) if micr_total_comparable_count > 0 else 0
    print(f"MICR Code Match Accuracy: {micr_accuracy:.2f}% ({micr_matches}/{micr_total_comparable_count} comparable pairs)")

print("--- End of Optimized Notebook ---")

In [ ]:
# Excel Comparison Analysis
# This notebook compares two Excel files cell by cell and calculates match percentages

import pandas as pd
import numpy as np
import re
from pathlib import Path
import os

# 1. Load the Excel files
def load_excel_files(extract_file_path, gold_file_path):
    """
    Load both Excel files into pandas DataFrames
    
    Parameters:
    extract_file_path (str): Path to the extract Excel file
    gold_file_path (str): Path to the gold Excel file
    
    Returns:
    tuple: (extract_df, gold_df) - Pandas DataFrames containing the data
    """
    print("Loading Excel files...")
    extract_df = pd.read_excel(extract_file_path)
    gold_df = pd.read_excel(gold_file_path)
    
    print(f"Extract Excel shape: {extract_df.shape}")
    print(f"Gold Excel shape: {gold_df.shape}")
    
    return extract_df, gold_df

# 2. Process and compare Excel files
def process_and_compare(extract_df, gold_df):
    """
    Process and compare the two Excel files.
    Note: It's expected that extract_df will have fewer rows than gold_df.
    The comparison is done based on records in extract_df, looking for matches in gold_df.
    
    Parameters:
    extract_df (DataFrame): Extract Excel DataFrame (will have fewer rows)
    gold_df (DataFrame): Gold Excel DataFrame (will have more rows)
    
    Returns:
    tuple: (results_df, summary_df) - DataFrames with comparison results and summary
    """
    # Function to extract INSTRUMENT_ID from filepath
    def extract_instrument_id(filepath):
        # Extract the filename without extension
        filename = os.path.basename(filepath)
        # Remove file extension and '_f' suffix
        instrument_id = filename.split('_')[0]
        return instrument_id
    
    # Function to construct MICR code from Gold Excel
    def construct_micr_code(row):
        """
        Construct MICR code by concatenating specified fields from Gold Excel
        Skip SCAN_MICR_ACNO if it's 0
        """
        components = []
        
        # Add components only if they exist and are not null
        if pd.notna(row['SCAN_INSTRUMENT_NUMBER']):
            components.append(str(row['SCAN_INSTRUMENT_NUMBER']))
        
        if pd.notna(row['SCAN_PAYEE_BANK_CITY_CODE']):
            components.append(str(row['SCAN_PAYEE_BANK_CITY_CODE']))
            
        if pd.notna(row['SCAN_PAYEE_BANK_CODE']):
            components.append(str(row['SCAN_PAYEE_BANK_CODE']))
            
        if pd.notna(row['SCAN_PAYEE_BANK_BRANCH_CODE']):
            components.append(str(row['SCAN_PAYEE_BANK_BRANCH_CODE']))
        
        # Only add SCAN_MICR_ACNO if it's not 0
        if pd.notna(row['SCAN_MICR_ACNO']) and row['SCAN_MICR_ACNO'] != 0:
            components.append(str(row['SCAN_MICR_ACNO']))
            
        if pd.notna(row['SCAN_INSTRUMENT_TYPE']):
            components.append(str(row['SCAN_INSTRUMENT_TYPE']))
        
        return ''.join(components)
    
    # Function to normalize text for comparison
    def normalize_text(text):
        """
        Normalize text by removing leading/trailing spaces, converting to lowercase,
        and removing special characters
        """
        if pd.isna(text):
            return ""
        
        # Convert to string if not already
        text = str(text)
        
        # Convert to lowercase and remove leading/trailing whitespace
        text = text.lower().strip()
        
        # Remove special characters
        text = re.sub(r'[^\w\s]', '', text)
        
        return text
    
    # Function to compare amounts with proper handling of decimal formats
    def compare_amounts(amount1, amount2):
        """
        Compare two amounts, handling different decimal formats
        """
        # Handle NaN values
        if pd.isna(amount1) or pd.isna(amount2):
            return pd.isna(amount1) and pd.isna(amount2)
        
        # Convert to float for comparison
        try:
            float_amount1 = float(amount1)
            float_amount2 = float(amount2)
            
            # Compare with a small tolerance for floating point errors
            return abs(float_amount1 - float_amount2) < 1e-6
        except (ValueError, TypeError):
            # If conversion fails, compare as strings
            return str(amount1) == str(amount2)
    
    print("Processing and comparing Excel files...")
    
    # Extract INSTRUMENT_ID from filepath in extract_df
    extract_df['extracted_instrument_id'] = extract_df['filepath'].apply(extract_instrument_id)
    
    # Construct MICR code in gold_df
    gold_df['constructed_micr_code'] = gold_df.apply(construct_micr_code, axis=1)
    
    # Create a results DataFrame
    results = []
    
    # Track overall matches
    total_records = len(extract_df)
    matched_records = 0
    
    # Track field-specific matches
    instrument_id_matches = 0
    micr_code_matches = 0
    payee_name_matches = 0
    amount_matches = 0
    
    # Print row counts to verify
    print(f"Extract Excel row count: {len(extract_df)}")
    print(f"Gold Excel row count: {len(gold_df)}")
    print(f"Note: Extract Excel has fewer rows as expected")
    
    # Compare each record in extract_df with corresponding record in gold_df
    for _, extract_row in extract_df.iterrows():
        extract_instrument_id = extract_row['extracted_instrument_id']
        
        # Find matching row in gold_df
        matching_gold_rows = gold_df[gold_df['INSTRUMENT_ID'] == extract_instrument_id]
        
        if len(matching_gold_rows) > 0:
            # Match found
            gold_row = matching_gold_rows.iloc[0]
            
            # Compare individual fields
            # 1. INSTRUMENT_ID match
            instrument_id_match = extract_instrument_id == gold_row['INSTRUMENT_ID']
            if instrument_id_match:
                instrument_id_matches += 1
            
            # 2. MICR code match
            micr_code_match = extract_row['micr_code'] == gold_row['constructed_micr_code']
            if micr_code_match:
                micr_code_matches += 1
            
            # 3. Payee name match (normalized)
            normalized_extract_payee = normalize_text(extract_row['payee_name'])
            normalized_gold_payee = normalize_text(gold_row['PRES_NAME'])
            payee_name_match = normalized_extract_payee == normalized_gold_payee
            if payee_name_match:
                payee_name_matches += 1
            
            # 4. Amount match
            amount_match = compare_amounts(extract_row['amount_numeric'], gold_row['CAR_AMOUNT'])
            if amount_match:
                amount_matches += 1
            
            # Calculate overall match for this record
            field_matches = [instrument_id_match, micr_code_match, payee_name_match, amount_match]
            record_match_percentage = sum(field_matches) / len(field_matches) * 100
            
            # If all fields match, increment matched_records
            if all(field_matches):
                matched_records += 1
            
            # Add to results
            results.append({
                'Extract_Filepath': extract_row['filepath'],
                'Extract_Instrument_ID': extract_instrument_id,
                'Gold_Instrument_ID': gold_row['INSTRUMENT_ID'],
                'Extract_MICR_Code': extract_row['micr_code'],
                'Gold_MICR_Code': gold_row['constructed_micr_code'],
                'MICR_Code_Match': micr_code_match,
                'Extract_Payee_Name': extract_row['payee_name'],
                'Gold_Payee_Name': gold_row['PRES_NAME'],
                'Payee_Name_Match': payee_name_match,
                'Extract_Amount': extract_row['amount_numeric'],
                'Gold_Amount': gold_row['CAR_AMOUNT'],
                'Amount_Match': amount_match,
                'Record_Match_Percentage': record_match_percentage
            })
        else:
            # No match found
            results.append({
                'Extract_Filepath': extract_row['filepath'],
                'Extract_Instrument_ID': extract_instrument_id,
                'Gold_Instrument_ID': 'No match found',
                'Extract_MICR_Code': extract_row['micr_code'],
                'Gold_MICR_Code': 'No match found',
                'MICR_Code_Match': False,
                'Extract_Payee_Name': extract_row['payee_name'],
                'Gold_Payee_Name': 'No match found',
                'Payee_Name_Match': False,
                'Extract_Amount': extract_row['amount_numeric'],
                'Gold_Amount': 'No match found',
                'Amount_Match': False,
                'Record_Match_Percentage': 0.0
            })
    
    # Create results DataFrame
    results_df = pd.DataFrame(results)
    
    # Calculate overall match percentages
    if total_records > 0:
        overall_match_percentage = (matched_records / total_records) * 100
        instrument_id_match_percentage = (instrument_id_matches / total_records) * 100
        micr_code_match_percentage = (micr_code_matches / total_records) * 100
        payee_name_match_percentage = (payee_name_matches / total_records) * 100
        amount_match_percentage = (amount_matches / total_records) * 100
    else:
        overall_match_percentage = 0
        instrument_id_match_percentage = 0
        micr_code_match_percentage = 0
        payee_name_match_percentage = 0
        amount_match_percentage = 0
    
    # Add summary statistics
    summary_df = pd.DataFrame([{
        'Metric': 'Overall Match',
        'Match_Percentage': overall_match_percentage,
        'Matched_Count': matched_records,
        'Total_Count': total_records
    }, {
        'Metric': 'Instrument ID Match',
        'Match_Percentage': instrument_id_match_percentage,
        'Matched_Count': instrument_id_matches,
        'Total_Count': total_records
    }, {
        'Metric': 'MICR Code Match',
        'Match_Percentage': micr_code_match_percentage,
        'Matched_Count': micr_code_matches,
        'Total_Count': total_records
    }, {
        'Metric': 'Payee Name Match',
        'Match_Percentage': payee_name_match_percentage,
        'Matched_Count': payee_name_matches,
        'Total_Count': total_records
    }, {
        'Metric': 'Amount Match',
        'Match_Percentage': amount_match_percentage,
        'Matched_Count': amount_matches,
        'Total_Count': total_records
    }])
    
    return results_df, summary_df

# 3. Save results to Excel
def save_results(results_df, summary_df, output_path):
    """
    Save comparison results to Excel file
    
    Parameters:
    results_df (DataFrame): Detailed comparison results
    summary_df (DataFrame): Summary statistics
    output_path (str): Path to save output Excel
    
    Returns:
    str: Path to saved Excel file
    """
    print(f"Saving results to {output_path}...")
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # Write detailed results to first sheet
        results_df.to_excel(writer, sheet_name='Detailed Results', index=False)
        
        # Write summary to second sheet
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    print(f"Results saved to {output_path}")
    return output_path

# 4. Main function to run the entire process
def main():
    """
    Main function to run the Excel comparison process
    """
    # Define file paths (update these to your actual file paths)
    extract_file_path = "extract.xlsx"  # The extract Excel (with fewer rows)
    gold_file_path = "gold.xlsx"        # The gold Excel (with more rows)
    output_path = "excel_comparison_results.xlsx"
    
    # Load Excel files
    extract_df, gold_df = load_excel_files(extract_file_path, gold_file_path)
    
    # Process and compare Excel files
    results_df, summary_df = process_and_compare(extract_df, gold_df)
    
    # Save results
    save_results(results_df, summary_df, output_path)
    
    print("Comparison completed successfully!")
    print(f"Overall match percentage: {summary_df.loc[0, 'Match_Percentage']:.2f}%")
    
    return results_df, summary_df

# Run the comparison if this script is executed directly
if __name__ == "__main__":
    results_df, summary_df = main()
    
    # Display summary
    print("\nSummary Statistics:")
    print(summary_df)